In [3]:
import os
import cv2
from pathlib import Path

# Paths
BASE_FOLDER=os.getcwd()
print(BASE_FOLDER)
UNIQUE_VIDEOS_FOLDER = BASE_FOLDER+"/unique_videos"
UNIQUE_PERFORMANCE_FOLDER = BASE_FOLDER+"/unique_performances"
INFLUENCER_FACE_DATA = BASE_FOLDER+"/influencer_face_data"
FACE_IMG_FOLDER = os.path.join(INFLUENCER_FACE_DATA, "face_img")
FACE_SCORE_FOLDER = os.path.join(INFLUENCER_FACE_DATA, "face_score")

# Create directories if they don't exist
os.makedirs(FACE_IMG_FOLDER, exist_ok=True)
os.makedirs(FACE_SCORE_FOLDER, exist_ok=True)

# Load a pre-trained face detection model (Haar cascade for simplicity)
face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + "haarcascade_frontalface_default.xml")

# Process each video in unique_videos
for video_file in os.listdir(UNIQUE_VIDEOS_FOLDER):
    video_path = os.path.join(UNIQUE_VIDEOS_FOLDER, video_file)
    performance_file = os.path.join(UNIQUE_PERFORMANCE_FOLDER, f"{Path(video_file).stem}.txt")
    
    # Skip if performance file doesn't exist
    if not os.path.exists(performance_file):
        print(f"Performance file missing for {video_file}. Skipping...")
        continue

    # Read the performance score
    with open(performance_file, "r") as pf:
        performance_score = pf.read().strip()

    # Open video file
    cap = cv2.VideoCapture(video_path)
    frame_count = 0
    face_count = 0

    while True:
        ret, frame = cap.read()
        if not ret:
            break

        frame_count += 1

        # Convert frame to grayscale for face detection
        gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

        # Detect faces
        faces = face_cascade.detectMultiScale(gray, scaleFactor=1.1, minNeighbors=5, minSize=(30, 30))

        for (x, y, w, h) in faces:
            # Extract face from frame
            face = frame[y:y+h, x:x+w]

            # Save face image
            face_img_name = f"{Path(video_file).stem}_frame{frame_count}_face{face_count}.jpg"
            face_img_path = os.path.join(FACE_IMG_FOLDER, face_img_name)
            cv2.imwrite(face_img_path, face)

            # Save performance score
            face_score_path = os.path.join(FACE_SCORE_FOLDER, f"{Path(face_img_name).stem}.txt")
            with open(face_score_path, "w") as sf:
                sf.write(performance_score)

            face_count += 1

    cap.release()

print("Face detection and extraction completed!")


/home/vishist/coding/Startup_Assignments/FuelGrowth/Influencer-Face-Metrics/Jupyter_notebooks
Face detection and extraction completed!


In [ ]:
import os
import cv2
import face_recognition
from pathlib import Path
from scipy.spatial.distance import cosine

# Paths
BASE_FOLDER=os.getcwd()
print(BASE_FOLDER)
UNIQUE_VIDEOS_FOLDER = BASE_FOLDER+"/unique_videos"
UNIQUE_PERFORMANCE_FOLDER = BASE_FOLDER+"/unique_performances"
INFLUENCER_FACE_DATA = BASE_FOLDER+"/influencer_face_data"
FACE_IMG_FOLDER = os.path.join(INFLUENCER_FACE_DATA, "face_img")
FACE_SCORE_FOLDER = os.path.join(INFLUENCER_FACE_DATA, "face_score")

# Create directories if they don't exist
os.makedirs(FACE_IMG_FOLDER, exist_ok=True)
os.makedirs(FACE_SCORE_FOLDER, exist_ok=True)

# Parameters
SIMILARITY_THRESHOLD = 0.6  # Threshold for face similarity (lower is stricter)

# To store embeddings of saved faces
saved_face_embeddings = []

# Function to check if a face is unique
def is_unique_face(new_embedding, saved_embeddings, threshold=SIMILARITY_THRESHOLD):
    for embedding in saved_embeddings:
        distance = cosine(embedding, new_embedding)
        if distance < threshold:
            return False  # Not unique
    return True  # Unique

# Process each video in unique_videos
for video_file in os.listdir(UNIQUE_VIDEOS_FOLDER):
    video_path = os.path.join(UNIQUE_VIDEOS_FOLDER, video_file)
    performance_file = os.path.join(UNIQUE_PERFORMANCE_FOLDER, f"{Path(video_file).stem}.txt")
    
    # Skip if performance file doesn't exist
    if not os.path.exists(performance_file):
        print(f"Performance file missing for {video_file}. Skipping...")
        continue

    # Read the performance score
    with open(performance_file, "r") as pf:
        performance_score = pf.read().strip()

    # Open video file
    cap = cv2.VideoCapture(video_path)
    frame_count = 0
    face_count = 0

    while True:
        ret, frame = cap.read()
        if not ret:
            break

        frame_count += 1

        # Detect faces using face_recognition
        face_locations = face_recognition.face_locations(frame)  # Detect face bounding boxes
        face_encodings = face_recognition.face_encodings(frame, face_locations)  # Compute embeddings

        for face_location, face_encoding in zip(face_locations, face_encodings):
            # Check if the face is unique
            if not is_unique_face(face_encoding, saved_face_embeddings):
                continue  # Skip similar face

            # Save the face embedding to the list
            saved_face_embeddings.append(face_encoding)

            # Save the face image
            top, right, bottom, left = face_location
            face = frame[top:bottom, left:right]
            face_img_name = f"{Path(video_file).stem}_frame{frame_count}_face{face_count}.jpg"
            face_img_path = os.path.join(FACE_IMG_FOLDER, face_img_name)
            cv2.imwrite(face_img_path, face)

            # Save performance score
            face_score_path = os.path.join(FACE_SCORE_FOLDER, f"{Path(face_img_name).stem}.txt")
            with open(face_score_path, "w") as sf:
                sf.write(performance_score)

            face_count += 1

    cap.release()

print("Face detection and extraction completed!")


In [1]:
!pip install ultralytics

  Using cached torch-2.5.1-cp312-cp312-manylinux1_x86_64.whl.metadata (28 kB)
  Using cached nvidia_cuda_nvrtc_cu12-12.4.127-py3-none-manylinux2014_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_runtime_cu12-12.4.127-py3-none-manylinux2014_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_cupti_cu12-12.4.127-py3-none-manylinux2014_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cudnn_cu12-9.1.0.70-py3-none-manylinux2014_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cublas_cu12-12.4.5.8-py3-none-manylinux2014_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cufft_cu12-11.2.1.3-py3-none-manylinux2014_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_curand_cu12-10.3.5.147-py3-none-manylinux2014_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cusolver_cu12-11.6.1.9-py3-none-manylinux2014_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cusparse_cu12-12.3.1.170-py3-none-manylinux2014_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_nccl_cu12-2.21.5-py3-none

In [5]:
import os
import cv2
from pathlib import Path
from ultralytics import YOLO

BASE_FOLDER=os.getcwd()
print(BASE_FOLDER)
UNIQUE_VIDEOS_FOLDER = BASE_FOLDER+"/unique_videos"
UNIQUE_PERFORMANCE_FOLDER = BASE_FOLDER+"/unique_performances"
INFLUENCER_FACE_DATA = BASE_FOLDER+"/influencer_face_data"
FACE_IMG_FOLDER = os.path.join(INFLUENCER_FACE_DATA, "face_img")
FACE_SCORE_FOLDER = os.path.join(INFLUENCER_FACE_DATA, "face_score")

os.makedirs(FACE_IMG_FOLDER, exist_ok=True)
os.makedirs(FACE_SCORE_FOLDER, exist_ok=True)

model = YOLO("yolov8n.pt")

for video_file in os.listdir(UNIQUE_VIDEOS_FOLDER):
    video_path = os.path.join(UNIQUE_VIDEOS_FOLDER, video_file)
    performance_file = os.path.join(UNIQUE_PERFORMANCE_FOLDER, f"{Path(video_file).stem}.txt")
    
    if not os.path.exists(performance_file):
        print(f"Performance file missing for {video_file}. Skipping...")
        continue

    with open(performance_file, "r") as pf:
        performance_score = pf.read().strip()

    cap = cv2.VideoCapture(video_path)
    frame_count = 0
    face_count = 0

    while True:
        ret, frame = cap.read()
        if not ret:
            break

        frame_count += 1
        if frame_count % 10 != 0:
            continue

        results = model.predict(frame, conf=0.5, verbose=False)
        detections = results[0].boxes.xyxy

        for detection in detections:
            x1, y1, x2, y2 = map(int, detection)
            face = frame[y1:y2, x1:x2]

            face_img_name = f"{Path(video_file).stem}_frame{frame_count}_face{face_count}.jpg"
            face_img_path = os.path.join(FACE_IMG_FOLDER, face_img_name)
            cv2.imwrite(face_img_path, face)

            face_score_path = os.path.join(FACE_SCORE_FOLDER, f"{Path(face_img_name).stem}.txt")
            with open(face_score_path, "w") as sf:
                sf.write(performance_score)

            face_count += 1

    cap.release()

print("Face detection and extraction completed!")


/home/vishist/coding/Startup_Assignments/FuelGrowth/Influencer-Face-Metrics/Jupyter_notebooks
Face detection and extraction completed!


In [10]:
import os
import cv2
import dlib
from pathlib import Path


BASE_FOLDER=os.getcwd()
print(BASE_FOLDER)
UNIQUE_VIDEOS_FOLDER = BASE_FOLDER+"/unique_videos"
UNIQUE_PERFORMANCE_FOLDER = BASE_FOLDER+"/unique_performances"
INFLUENCER_FACE_DATA = BASE_FOLDER+"/influencer_face_data"
FACE_IMG_FOLDER = os.path.join(INFLUENCER_FACE_DATA, "face_img")
FACE_SCORE_FOLDER = os.path.join(INFLUENCER_FACE_DATA, "face_score")

os.makedirs(FACE_IMG_FOLDER, exist_ok=True)
os.makedirs(FACE_SCORE_FOLDER, exist_ok=True)

face_detector = dlib.get_frontal_face_detector()

for video_file in os.listdir(UNIQUE_VIDEOS_FOLDER):
    video_path = os.path.join(UNIQUE_VIDEOS_FOLDER, video_file)
    performance_file = os.path.join(UNIQUE_PERFORMANCE_FOLDER, f"{Path(video_file).stem}.txt")
    
    if not os.path.exists(performance_file):
        print(f"Performance file missing for {video_file}. Skipping...")
        continue

    with open(performance_file, "r") as pf:
        performance_score = pf.read().strip()

    cap = cv2.VideoCapture(video_path)
    frame_count = 0
    face_count = 0

    while True:
        ret, frame = cap.read()
        if not ret:
            break

        frame_count += 1
        if frame_count % 10 != 0:
            continue

        gray_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

        faces = face_detector(gray_frame)

        for face in faces:
            h, w, _ = frame.shape
            x1 = max(0, face.left())
            y1 = max(0, face.top())
            x2 = min(w, face.right())
            y2 = min(h, face.bottom())

            if x1 < x2 and y1 < y2:
                face_crop = frame[y1:y2, x1:x2]

                face_img_name = f"{Path(video_file).stem}_frame{frame_count}_face{face_count}.jpg"
                face_img_path = os.path.join(FACE_IMG_FOLDER, face_img_name)
                cv2.imwrite(face_img_path, face_crop)

                face_score_path = os.path.join(FACE_SCORE_FOLDER, f"{Path(face_img_name).stem}.txt")
                with open(face_score_path, "w") as sf:
                    sf.write(performance_score)

                face_count += 1

    cap.release()

print("Face detection and extraction completed!")


/home/vishist/coding/Startup_Assignments/FuelGrowth/Influencer-Face-Metrics/Jupyter_notebooks
Face detection and extraction completed!
